# Import

In [43]:
from stonks import data_handler
# import stonks

In [2]:
data_handler_obj = data_handler()
data_handler_obj.download_data()

Download bitch.


'blah'

In [3]:
data_handler_obj.download_data()

Download bitch.


'blah'

In [19]:
data_handler_obj.temp_data

AttributeError: 'data_handler' object has no attribute 'temp_data'

In [1]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io
from tqdm import tqdm
import time
import pandas as pd
import pathlib

# Different Libraries to get Historical Data

After a bunch of digging I'm giving up on finding the SecurityIds(ticker symbols). I'll deal with the ticker IDs and use the BSE data tool to get the ticker symbol whenenver I need it.

## bsedata

Problem is there is an error
InvalidStockException: Suspended due to Procedural reasons

### get symbol from tickerID

In [5]:
from bsedata.bse import BSE

def initialise_symbol_converter(b = None):
    b = BSE()
    print(b)
    # Output:
    # Driver Class for Bombay Stock Exchange (BSE)

    # to execute "updateScripCodes" on instantiation
    b = BSE(update_codes = True)
    return b

def get_symbol_from_ticker_ID(tickerID, b = None):
    tickerID =''.join(i for i in tickerID if i.isdigit())
    if b is None:
        b = initialise_symbol_converter(b) # This b business is weird. It'll make more sense in a class.
    quote = b.getQuote(tickerID)
    company_ID = quote["securityID"]
    return company_ID

get_symbol_from_ticker_ID("BOM532057")

Driver Class for Bombay Stock Exchange (BSE)


IndexError: list index out of range

In [6]:
b = BSE()
print(b)
# Output:
# Driver Class for Bombay Stock Exchange (BSE)

# to execute "updateScripCodes" on instantiation
b = BSE(update_codes = True)

Driver Class for Bombay Stock Exchange (BSE)


In [11]:
scrip_codes_to_company_names = b.getScripCodes() # All bse scrip codes and company names
print(len(list(scrip_codes_to_company_names.keys())), "scrip codes")

4487 scrip codes


In [12]:
scrip_codes_to_company_names

{'533022': '20 Microns Ltd.',
 '532628': '3i Infotech Ltd.',
 '523395': '3m India Ltd.',
 '590116': '7seas Technologies Ltd-$',
 '512161': '8k Miles Software Services Ltd.',
 '538351': 'A.f. Enterprises Ltd',
 '539300': 'A.k. Spintex Ltd',
 '530499': 'A.k.capital Services Ltd.',
 '533292': 'A2z Infra Engineering Limited',
 '531611': 'Aadhaar Ventures India Ltd.',
 '530027': 'Aadi Industries Ltd.',
 '531866': 'Aagam Capital Ltd.',
 '539096': 'Aananda Lakshmi Spinning Mills Ltd',
 '538812': 'Aanchal Ispat Ltd',
 '524412': 'Aarey Drugs & Pharmaceuticals Ltd.',
 '524348': 'Aarti Drugs Ltd.',
 '524208': 'Aarti Industries Ltd.',
 '514274': 'Aarvee Denims & Exports Ltd.',
 '531731': 'Aarya Global Shares And Securities Ltd.',
 '519319': 'Aashee Infotech Ltd.',
 '512038': 'Aaswa Trading & Exports Ltd.',
 '523204': 'Aban Offshore Ltd.',
 '512165': 'Abans Enterprises Ltd',
 '500002': 'Abb India Limited',
 '500488': 'Abbott India Ltd.',
 '505665': 'Abc Bearings Ltd.-$',
 '513119': 'Abc Gas (intern

In [9]:
def get_all_securityIDs():
    all_IDs = {}
    scrip_codes_to_company_names = b.getScripCodes() # All bse scrip codes and company names
    problematic_scrip_ids = []
    for key in tqdm(scrip_codes_to_company_names.keys()):
        try:
            temp_quote = b.getQuote(key) 
        except KeyboardInterrupt:
            print('Interrupted')
            return
        except:
            problematic_scrip_ids.append(key)
        company_security_id = temp_quote["securityID"]
        all_IDs[company_security_id] = {
            "scripCode" : key,
            "companyName" : temp_quote["companyName"]
        }
#         time.sleep(1)
    return all_IDs, problematic_scrip_ids

In [7]:
company_ids_and_data = get_all_securityIDs()

  0%|          | 19/4487 [00:07<27:44,  2.68it/s]

Interrupted


## quandl 

Looks like free data and an API?

https://www.quandl.com/

5R9HxL17xSYKQn31UMX7 is the API key

Authenticated users have a limit of 300 calls per 10 seconds, 2,000 calls per 10 minutes and a limit of 50,000 calls per day.

Authenticated users of free datasets have a concurrency limit of one; that is, they can make one call at a time and have an additional call in the queue.

Premium data subscribers have a limit of 5,000 calls per 10 minutes and a limit of 720,000 calls per day.

Lmao 2500 $ per month 

https://www.quandl.com/databases/SLD/pricing/plans

Free data doesn't have a lot of stuff, for example BHEL.

But this is free : https://www.quandl.com/data/BSE-Bombay-Stock-Exchange

### Reading the BSE data

From clicking on the "stocks" link on :
https://www.quandl.com/data/BSE-Bombay-Stock-Exchange/documentation

In [14]:
stocks_df = pd.read_table("data/all_tickerIds_from_quandl.txt", delimiter="|")

In [15]:
stocks_df

,STOCK,CODE
0,20 Microns Ltd. EOD Prices,BOM533022
1,3i Infotech Ltd. EOD Prices,BOM532628
2,3m India Ltd. EOD Prices,BOM523395
3,7seas Technologies Ltd-$ EOD Prices,BOM590116
4,8k Miles Software Services Ltd. EOD Prices,BOM512161
...,...,...
4534,Zuari Agro Chemicals Ltd. EOD Prices,BOM534742
4535,Zuari Global Ltd. EOD Prices,BOM500780
4536,Zyden Gentec Ltd. EOD Prices,BOM530091
4537,Zydus Wellness Ltd.-$ EOD Prices,BOM531335


In [13]:
n = 0
for index, row in stocks_df.iterrows():
    if "EOD Prices" not in row["STOCK"]: 
        print(row["STOCK"])
        n += 1
print(n)
# No idea what this is, seems pretty useless. Need to dig later.

BSE 100
BSE 200
BSE 500 Shariah
BSE 500
BSE AUTO
BSE AllCap
BSE BANKEX
BSE Basic Materials
BSE CAPITAL GOODS
BSE CARBONEX
BSE CONSUMER DURABLES
BSE CPSE
BSE Consumer Discretionary Goods and Services
BSE DOLLEX 100
BSE DOLLEX 200
BSE DOLLEX 30
BSE Dividend Stability Index
BSE Energy
BSE Enhanced Value Index
BSE Fast Moving Consumer Goods
BSE Finance
BSE GREENEX
BSE Healthcare
BSE IPO
BSE India 10 Year Sovereign Bond
BSE India Infrastructure Index
BSE India Manufacturing Index
BSE Industrials
BSE Information Technology
BSE LargeCap
BSE LargeMidCap
BSE Low Volatility Index
BSE METAL
BSE MidCap Select Index
BSE MidCap
BSE MidSmallCap
BSE Momentum Index
BSE OIL and GAS
BSE POWER
BSE PSU
BSE Quality Index
BSE REALTY
BSE SENSEX Dynamic Gold Hedged Index
BSE SENSEX Gold Hedged Index
BSE SENSEX
BSE SME IPO
BSE SmallCap Select Index
BSE SmallCap
BSE TECK
BSE Telecom
BSE Utilities
BlackRock NY Muni Inc Qty Common (BSE) Stock Prices, Dividends and Splits
52


### Api calls for data

https://docs.quandl.com/docs/getting-started

In [23]:
stocks_df.shape, stocks_df.head()

((4539, 2),
                                         STOCK       CODE
 0                  20 Microns Ltd. EOD Prices  BOM533022
 1                 3i Infotech Ltd. EOD Prices  BOM532628
 2                    3m India Ltd. EOD Prices  BOM523395
 3         7seas Technologies Ltd-$ EOD Prices  BOM590116
 4  8k Miles Software Services Ltd. EOD Prices  BOM512161)

In [17]:
import quandl
quandl.ApiConfig.api_key = "5R9HxL17xSYKQn31UMX7"

In [19]:
quandl_location

'BSE/BOM533022'

In [33]:
ticker_to_look_into = "BOM533022"
quandl_location = 'BSE/'+ticker_to_look_into

data = quandl.get(quandl_location) # IDK what this is 

In [37]:
path = pathlib.Path("./data/quandl/"+quandl_location+".csv")
path.parent.mkdir(parents=True, exist_ok=True)

In [36]:
path.parent

PosixPath('data/quandl/BSE')

In [39]:
data.to_csv("./data/quandl/"+quandl_location+".csv")

In [25]:
stocks_df.head()

,STOCK,CODE
0,20 Microns Ltd. EOD Prices,BOM533022
1,3i Infotech Ltd. EOD Prices,BOM532628
2,3m India Ltd. EOD Prices,BOM523395
3,7seas Technologies Ltd-$ EOD Prices,BOM590116
4,8k Miles Software Services Ltd. EOD Prices,BOM512161


In [40]:
def download_single_quandl_data_and_save(directory, dataID):
    quandl_location = directory+"/"+dataID
    downloaded_data = quandl.get(quandl_location)
    path_to_save_data = pathlib.Path("./data/quandl/"+quandl_location+".csv")
    path_to_save_data.parent.mkdir(parents=True, exist_ok=True)
    downloaded_data.to_csv(path_to_save_data)
    print("Sucessfully downloaded and saved : ", quandl_location)

In [54]:
def append_broken_code_to_failed_download_text_file(download_source, download_data_code):
    actual_text_file_path = "./data/" + download_source + "/failed_downloads.txt"
    with open(actual_text_file_path, "a+") as file_object:
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0 :
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(download_data_code)
        

In [56]:
directory = "BSE"
for _, row in tqdm(stocks_df.iterrows()):
    try:
        download_single_quandl_data_and_save(directory=directory, dataID=row["CODE"])
    except KeyboardInterrupt:
        print('Interrupted')
    except:
        print("Something failed. Code is :", row["CODE"]) # Append to text file
        append_broken_code_to_failed_download_text_file(download_source="quandl", download_data_code = row["CODE"])
    break

0it [00:01, ?it/s]

Something failed. Code is : BOM533022


In [53]:
with open("data/quandl/failed_quandl_codes.txt", "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    # If file is not empty then append '\n'
    data = file_object.read(100)
    if len(data) > 0 :
        file_object.write("\n")
    # Append text at the end of file
    file_object.write("hello hi a new code should be here.")

In [33]:
all_data.keys()

dict_keys(['BOM533022', 'BOM532628', 'BOM523395', 'BOM590116', 'BOM512161', 'BOM538351', 'BOM539300', 'BOM530499', 'BOM533292', 'BOM531611', 'BOM530027', 'BOM531866', 'BOM539096', 'BOM538812', 'BOM524412', 'BOM524348', 'BOM524208', 'BOM514274', 'BOM531731', 'BOM519319', 'BOM512038', 'BOM523204', 'BOM512165', 'BOM500002', 'BOM500488', 'BOM505665', 'BOM513119', 'BOM520123', 'BOM532682', 'BOM532057', 'BOM538952', 'BOM532831', 'BOM538935', 'BOM511756', 'BOM526955', 'BOM531161', 'BOM500410', 'BOM532774', 'BOM517494'])

In [35]:
all_data["BOM531731"]

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
Date,,,,,,,,,,,,
2005-08-31,11.00,11.00,11.00,11.00,11.00,100.0,1.0,1100.0,100.0,100.0,0.00,0.00
2005-09-20,13.20,13.20,13.20,13.20,13.20,100.0,1.0,1320.0,NaN,NaN,0.00,0.00
2008-03-03,12.60,12.60,12.60,12.60,12.60,100.0,1.0,1260.0,100.0,100.0,0.00,0.00
2008-03-05,12.00,12.00,12.00,12.00,12.00,200.0,1.0,2400.0,200.0,100.0,0.00,0.00
2008-03-07,12.60,12.60,12.60,12.60,12.60,200.0,1.0,2520.0,200.0,100.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-04-03,0.50,0.54,0.50,0.50,0.50,34400.0,20.0,17351.0,34400.0,100.0,0.04,0.00
2017-04-10,0.48,0.48,0.48,0.48,0.48,4580.0,7.0,2198.0,4580.0,100.0,0.00,0.00
2017-04-17,0.46,0.46,0.46,0.46,0.46,1912.0,5.0,879.0,1912.0,100.0,0.00,0.00


In [29]:
print(data.columns)

data # This data looks gorgeous. Need to figure out what the columns are.

Index(['Open', 'High', 'Low', 'Close', 'WAP', 'No. of Shares', 'No. of Trades',
       'Total Turnover', 'Deliverable Quantity', '% Deli. Qty to Traded Qty',
       'Spread H-L', 'Spread C-O'],
      dtype='object')


,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
Date,,,,,,,,,,,,
2008-10-06,50.00,96.25,31.50,33.30,41.599186,12913246.0,61654.0,537180527.0,791756.0,6.13,64.75,-16.70
2008-10-07,33.70,37.95,27.65,30.40,31.188726,4371270.0,18368.0,136334341.0,342479.0,7.83,10.30,-3.30
2008-10-08,27.55,28.95,25.25,26.35,26.945326,1470611.0,5129.0,39626093.0,211928.0,14.41,3.70,-1.20
2008-10-10,22.00,24.90,21.50,23.35,23.617522,681356.0,2679.0,16091940.0,163079.0,23.93,3.40,1.35
2008-10-13,24.00,26.70,23.85,24.90,25.573681,439040.0,2184.0,11227869.0,102108.0,23.26,2.85,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,64.05,69.00,63.00,69.00,67.420000,155752.0,1508.0,10500873.0,91991.0,59.06,6.00,4.95
2021-05-25,71.75,74.80,68.40,69.95,71.610000,179063.0,2752.0,12822640.0,78699.0,43.95,6.40,-1.80
2021-05-26,72.75,72.90,67.55,68.25,70.570000,94579.0,1548.0,6673969.0,41991.0,44.40,5.35,-4.50


In [7]:
data

,Value
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87
...,...
2021-05-18,65.49
2021-05-19,63.28
2021-05-20,61.95


# Getting stock features like moving average and stuff

In [30]:
single_stock_df = data_handler.get_stock_data(ticker_code='BOM523395')

In [31]:
single_stock_df

,Date,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
0,1991-06-26,80.0,85.0,80.0,85.00,0.00,0.0,0.0,0.0,NaN,NaN,5.0,5.00
1,1991-06-27,77.5,82.5,77.5,80.00,0.00,0.0,0.0,0.0,NaN,NaN,5.0,2.50
2,1991-06-28,82.5,92.5,82.5,88.75,0.00,0.0,0.0,0.0,NaN,NaN,10.0,6.25
3,1991-07-02,87.5,88.5,87.5,88.50,0.00,0.0,0.0,0.0,NaN,NaN,1.0,1.00
4,1991-07-03,94.0,94.0,87.5,90.00,0.00,0.0,0.0,0.0,NaN,NaN,6.5,-4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7081,2021-05-24,26500.0,26500.0,26027.0,26090.35,26116.41,172.0,103.0,4492022.0,116.0,67.44,473.0,-409.65
7082,2021-05-25,26002.0,26537.2,25894.1,26405.85,26210.07,274.0,160.0,7181559.0,137.0,50.00,643.1,403.85
7083,2021-05-26,26490.0,26600.0,26175.3,26258.90,26301.85,194.0,142.0,5102558.0,116.0,59.79,424.7,-231.10
7084,2021-05-27,26452.2,26713.0,26131.9,26597.50,26324.05,189.0,105.0,4975246.0,57.0,30.16,581.1,145.30


In [14]:
len(scrip_codes_to_company_names.keys())

4487

In [4]:
# Help about this, 'ta', extension
help(df.ta)

Help on AnalysisIndicators in module pandas_ta.core object:

class AnalysisIndicators(BasePandasObject)
 |  AnalysisIndicators(df, **kwargs)
 |  
 |  AnalysisIndicators is a class that extends the Pandas DataFrame via
 |  Pandas @pd.api.extensions.register_dataframe_accessor('name') decorator.
 |  
 |  This Pandas Extension is named 'ta' for Technical Analysis that allows us
 |  to apply technical indicators by extension.  Even though 'ta' is a
 |  Pandas DataFrame Extension, you can still call the Indicators
 |  individually. Use help() if needed.
 |  
 |  By default the 'ta' extension uses lower case column names: open, high,
 |  low, close, and volume.  You can override the defaults by providing the
 |  it's replacement name when calling the indicator.  For example, to call the indicator hl2().
 |  
 |  With 'default' columns: open, high, low, close, and volume.
 |  >>> df.ta.hl2()
 |  >>> df.ta(kind="hl2")
 |  
 |  With DataFrame columns: Open, High, Low, Close, and Volume.
 |  >>>

In [3]:
# List of all indicators
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.2.45b0
Total Indicators: 133
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_doji, cdl_inside, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, decay, decreasing, dema, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, kama, kc, kdj, kst, kurtosis, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, ssf, stdev, stoch, stochrsi, supertrend, swma, t3, tema, thermo, trend_return, trima, trix, true_range, tsi, ttm_trend, ui, uo, variance, vidya, vortex, vp, vwap, vwma, wcp, willr, wma, zlm

In [11]:
single_stock_df.ta.log_return(cumulative=True, append=True)

0            NaN
1      -0.060625
2       0.043172
3       0.040351
4       0.057158
          ...   
7081    5.726670
7082    5.738690
7083    5.733109
7084    5.745921
7085    5.695313
Name: CUMLOGRET_1, Length: 7086, dtype: float64

In [45]:
ta

<module 'pandas_ta' from '/home/swagget/Programs/anaconda/envs/py37/lib/python3.7/site-packages/pandas_ta/__init__.py'>

In [44]:
new_indicator = ta.sma(single_stock_df["close"], length=2)
print(new_indicator)

0             NaN
1          82.500
2          84.375
3          88.625
4          89.250
          ...    
7081    26083.850
7082    26248.100
7083    26332.375
7084    26428.200
7085    25941.225
Name: SMA_2, Length: 7086, dtype: float64


In [41]:
ta.sma?

In [35]:
CustomStrategy = ta.Strategy(
    name="20-day simple moving average",
    description="Just looks at the 20 day simple moving average. This is just to test if the strategy is sound.",
    ta=[
        {"kind": "sma", "length": 2, "prefix" : "small_test", "col_names": ("this should be new")},
#         {"kind": "sma", "length": 200},
#         {"kind": "bbands", "length": 20},
#         {"kind": "rsi"},
#         {"kind": "macd", "fast": 8, "slow": 21},
#         {"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    ]
)
# To run your "Custom Strategy"
single_stock_df.ta.strategy(CustomStrategy)

In [36]:
single_stock_df

,date,open,high,low,close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O,small_test_SMA_2
0,1991-06-26,80.0,85.0,80.0,85.00,0.00,0.0,0.0,0.0,NaN,NaN,5.0,5.00,NaN
1,1991-06-27,77.5,82.5,77.5,80.00,0.00,0.0,0.0,0.0,NaN,NaN,5.0,2.50,82.500
2,1991-06-28,82.5,92.5,82.5,88.75,0.00,0.0,0.0,0.0,NaN,NaN,10.0,6.25,84.375
3,1991-07-02,87.5,88.5,87.5,88.50,0.00,0.0,0.0,0.0,NaN,NaN,1.0,1.00,88.625
4,1991-07-03,94.0,94.0,87.5,90.00,0.00,0.0,0.0,0.0,NaN,NaN,6.5,-4.00,89.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7081,2021-05-24,26500.0,26500.0,26027.0,26090.35,26116.41,172.0,103.0,4492022.0,116.0,67.44,473.0,-409.65,26083.850
7082,2021-05-25,26002.0,26537.2,25894.1,26405.85,26210.07,274.0,160.0,7181559.0,137.0,50.00,643.1,403.85,26248.100
7083,2021-05-26,26490.0,26600.0,26175.3,26258.90,26301.85,194.0,142.0,5102558.0,116.0,59.79,424.7,-231.10,26332.375
7084,2021-05-27,26452.2,26713.0,26131.9,26597.50,26324.05,189.0,105.0,4975246.0,57.0,30.16,581.1,145.30,26428.200


In [1]:
# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
df.set_index(pd.DatetimeIndex(df["datetime"]), inplace=True)

# Calculate Returns and append to the df DataFrame
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

AttributeError: 'AnalysisIndicators' object has no attribute 'ticker'

# Rough came from the net

US based so I didn't dig further.

https://towardsdatascience.com/downloading-historical-stock-prices-in-python-93f85f059c1f

In [5]:
url="https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv"
s = requests.get(url).content
companies = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [6]:
companies

,Symbol,Company Name,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size
0,AAIT,iShares MSCI All Country Asia Information Tech...,iShares MSCI All Country Asia Information Tech...,G,N,N,100.0
1,AAL,"American Airlines Group, Inc.","American Airlines Group, Inc. - Common Stock",Q,N,N,100.0
2,AAME,Atlantic American Corporation,Atlantic American Corporation - Common Stock,G,N,N,100.0
3,AAOI,"Applied Optoelectronics, Inc.","Applied Optoelectronics, Inc. - Common Stock",G,N,N,100.0
4,AAON,"AAON, Inc.","AAON, Inc. - Common Stock",Q,N,N,100.0
...,...,...,...,...,...,...,...
2962,ZN,Zion Oil & Gas Inc,Zion Oil & Gas Inc - Common Stock,G,N,N,100.0
2963,ZNGA,Zynga Inc.,Zynga Inc. - Class A Common Stock,Q,N,N,100.0
2964,ZSPH,"ZS Pharma, Inc.","ZS Pharma, Inc. - Common Stock",G,N,N,100.0
2965,ZU,"zulily, inc.","zulily, inc. - Class A Common Stock",Q,N,N,100.0
